> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [2]:
import pandas as pd

df = pd.read_csv("salaries.csv")
print(df)

     FIRST NAME   LAST NAME SEX         DOJ CURRENT DATE     DESIGNATION  \
0        TOMASA       ARMEN   F   5-18-2014   01-07-2016         Analyst   
1         ANNIE         NaN   F         NaN   01-07-2016       Associate   
2         OLIVE        ANCY   F   7-28-2014   01-07-2016         Analyst   
3        CHERRY     AQUILAR   F  04-03-2013   01-07-2016         Analyst   
4          LEON  ABOULAHOUD   M  11-20-2014   01-07-2016         Analyst   
...         ...         ...  ..         ...          ...             ...   
2634  KATHERINE      ALSDON   F   6-28-2011   01-07-2016  Senior Manager   
2635     LOUISE     ALTARAS   F   1-14-2014   01-07-2016         Analyst   
2636      RENEE      ALVINO   F   1-23-2014   01-07-2016         Analyst   
2637       TERI   ANASTASIO   F   3-17-2014   01-07-2016         Analyst   
2638    GREGORY      ABARCA   M   9-18-2014   01-07-2016         Analyst   

       AGE  SALARY        UNIT  LEAVES USED  LEAVES REMAINING  RATINGS  \
0     21.0   

#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [62]:
### Escreva sua resposta aqui
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, Enum, select, func, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import enum

Base = declarative_base()

class SexEnum(enum.Enum):
    F = "F"
    M = "M"

class DesignationEnum(enum.Enum):
    Analyst = "Analyst"
    SeniorAnalyst = "Senior Analyst"
    Manager = "Manager"
    Associate = "Associate"
    Director = "Director"
    SeniorManager = "Senior Manager"

class UnitEnum(enum.Enum):
    IT = "IT"
    Finance = "Finance"
    Marketing = "Marketing"

class Employee(Base):
    __tablename__ = "profissionais_dados"

    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String, nullable=False)
    last_name = Column(String, nullable=False)
    sex = Column(Enum(SexEnum), nullable=False)
    doj = Column(Date, nullable=False)
    current_date = Column(Date, nullable=False)
    # designation = Column(Enum(DesignationEnum, name = 'designationenum'), nullable = False)
    designation = Column(String, nullable = False)
    age = Column(Integer, nullable=False)
    salary = Column(Float, nullable=False)
    unit = Column(Enum(UnitEnum), nullable=False)
    leaves_used = Column(Integer, nullable=False)
    leaves_remaining = Column(Integer, nullable=False)
    ratings = Column(Float, nullable=False)
    past_exp = Column(Float, nullable=False)



C:\Users\PDBD008\AppData\Local\Temp\ipykernel_14492\748241095.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [4]:
engine = create_engine('sqlite:///salarios.db')

#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [5]:
### Escreva sua resposta aqui
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [6]:
### Escreva sua resposta aqui
df.to_sql('profissionais_dados', con=engine, if_exists='append', index=False)

2639

#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [63]:
### Execute aqui sua query SQL com SQLAlchemy


query = text("""
    SELECT DESIGNATION,
           MIN(SALARY / 12) AS min_monthly_salary,
           MAX(SALARY / 12) AS max_monthly_salary,
           AVG(SALARY / 12) AS avg_monthly_salary
    FROM profissionais_dados
    GROUP BY DESIGNATION;
"""
)

# Executando a query diretamente
with engine.connect() as connection:
    result = connection.execute(query)
    for row in result:
        print(row)

('Analyst', 3333, 4165, 3751.2170343766034)
('Associate', 5846, 8300, 7266.465408805031)
('Director', 17832, 32342, 23913.9375)
('Manager', 8343, 12407, 10522.246913580248)
('Senior Analyst', 4170, 5830, 4991.3258426966295)
('Senior Manager', 12614, 16631, 14888.225806451614)


In [64]:
### Execute aqui sua query SQL com SQLAlchemy + Pandas
query = """
SELECT DESIGNATION,
       MIN(SALARY) / 12 AS min_salary_monthly,
       MAX(SALARY) / 12 AS max_salary_monthly,
       AVG(SALARY) / 12 AS avg_salary_monthly
FROM profissionais_dados
GROUP BY DESIGNATION;
"""

df_result = pd.read_sql_query(query, engine)

print(df_result)

      DESIGNATION  min_salary_monthly  max_salary_monthly  avg_salary_monthly
0         Analyst                3333                4165         3751.675988
1       Associate                5846                8300         7266.915094
2        Director               17832               32342        23914.265625
3         Manager                8343               12407        10522.716049
4  Senior Analyst                4170                5830         4991.778792
5  Senior Manager               12614               16631        14888.689516


In [65]:
### Execute aqui sua query com SQLAlchemy ORM

Session = sessionmaker(bind=engine)
session = Session()

stmt = (
    select(
        Employee.designation,
        func.min(Employee.salary) / 12,
        func.max(Employee.salary) / 12,
        func.avg(Employee.salary) / 12
    )
    .group_by(Employee.designation)
)

result = session.execute(stmt)

rows = list(result)
if rows:
    for row in rows:
        print(f"Designation: {row[0]}, Min Salary (Monthly): {row[1]:.2f}, Max Salary (Monthly): {row[2]:.2f}, Avg Salary (Monthly): {row[3]:.2f}")
else:
    print("Nenhum dado foi retornado pela consulta.")

session.close()

Designation: Analyst, Min Salary (Monthly): 3333.42, Max Salary (Monthly): 4165.00, Avg Salary (Monthly): 3751.68
Designation: Associate, Min Salary (Monthly): 5846.17, Max Salary (Monthly): 8300.25, Avg Salary (Monthly): 7266.92
Designation: Director, Min Salary (Monthly): 17832.25, Max Salary (Monthly): 32342.67, Avg Salary (Monthly): 23914.27
Designation: Manager, Min Salary (Monthly): 8343.67, Max Salary (Monthly): 12407.50, Avg Salary (Monthly): 10522.72
Designation: Senior Analyst, Min Salary (Monthly): 4170.33, Max Salary (Monthly): 5830.50, Avg Salary (Monthly): 4991.78
Designation: Senior Manager, Min Salary (Monthly): 12614.42, Max Salary (Monthly): 16631.42, Avg Salary (Monthly): 14888.69
